Merge JSONL 

In [4]:
import json

files = [
    r"D:/chatbot_covid19_intern_procodecg/data/processed/covid_19_to_narative.jsonl",
    r"D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl"
]

merge_file = r"D:/chatbot_covid19_intern_procodecg/data/processed/dataset_covid__merge.jsonl"

with open(merge_file, "w", encoding="utf-8") as outfile:
    for path in files :
        with open(path, "r", encoding="utf-8") as infile:
            for line in infile:
                outfile.write(line)
                
print(f"semua chunk telah digabung di: {merge_file}")



semua chunk telah digabung di: D:/chatbot_covid19_intern_procodecg/data/processed/dataset_covid__merge.jsonl


In [1]:
from sentence_transformers import SentenceTransformer
import json
import numpy as np

chunks_path = r"D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl"
embedding_path = r"D:/chatbot_covid19_intern_procodecg/vector_db/faiss__bahasa_metadata.json"

# 1️⃣ Load model lokal untuk embedding
print("🧠 Memuat model SentenceTransformer ...")
model = SentenceTransformer("indobenchmark/indobert-base-p1")

embeddings = []

# 2️⃣ Baca dan buat embedding per chunk
with open(chunks_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        text = data["text"]

        emb = model.encode(text).tolist()
        data["embedding"] = emb

        embeddings.append(data)
        print(f"✅ Chunk {i+1} selesai di-embed")

# 3️⃣ Simpan hasilnya ke JSONL
with open(embedding_path, "w", encoding="utf-8") as f:
    for item in embeddings:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"💾 Semua embedding tersimpan di: {embedding_path}")


🧠 Memuat model SentenceTransformer ...


No sentence-transformers model found with name indobenchmark/indobert-base-p1. Creating a new one with MEAN pooling.


config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

d:\chatbot_covid19_intern_procodecg\.venvr\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


✅ Chunk 1 selesai di-embed
✅ Chunk 2 selesai di-embed
✅ Chunk 3 selesai di-embed
✅ Chunk 4 selesai di-embed
✅ Chunk 5 selesai di-embed
✅ Chunk 6 selesai di-embed
✅ Chunk 7 selesai di-embed
✅ Chunk 8 selesai di-embed
✅ Chunk 9 selesai di-embed
✅ Chunk 10 selesai di-embed
✅ Chunk 11 selesai di-embed
✅ Chunk 12 selesai di-embed
✅ Chunk 13 selesai di-embed
✅ Chunk 14 selesai di-embed
✅ Chunk 15 selesai di-embed
✅ Chunk 16 selesai di-embed
✅ Chunk 17 selesai di-embed
✅ Chunk 18 selesai di-embed
✅ Chunk 19 selesai di-embed
✅ Chunk 20 selesai di-embed
💾 Semua embedding tersimpan di: D:/chatbot_covid19_intern_procodecg/vector_db/faiss__bahasa_metadata.json


In [ ]:
from sentence_transformers import SentenceTransformer
import json
import numpy as np
import faiss
import os
from tqdm import tqdm

class COVID19EmbeddingGenerator:
    def __init__(self, model_name="thenlper/gte-base"):
        """
        Initialize embedding generator dengan model pilihan
        
        Model options:
        - "thenlper/gte-base" (Recommended)
        - "all-mpnet-base-v2"
        - "intfloat/multilingual-e5-base" 
        - "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
        - "all-MiniLM-L12-v2" (Lightweight)
        """
        print(f"🧠 Loading model: {model_name}")
        self.model = SentenceTransformer(model_name)
        self.model_name = model_name
        
    def load_chunks(self, chunks_path):
        """Load chunks dari JSONL file"""
        chunks = []
        with open(chunks_path, "r", encoding="utf-8") as f:
            for line in f:
                data = json.loads(line.strip())
                chunks.append(data)
        print(f"📖 Loaded {len(chunks)} chunks from {chunks_path}")
        return chunks
    
    def create_embeddings(self, chunks_path, output_path, batch_size=16):
        """Buat embeddings untuk semua chunks"""
        chunks = self.load_chunks(chunks_path)
        
        print("🔍 Creating embeddings...")
        all_embeddings = []
        
        for i in tqdm(range(0, len(chunks), batch_size), desc="Processing batches"):
            batch_chunks = chunks[i:i + batch_size]
            batch_texts = [chunk["text"] for chunk in batch_chunks]
            
            # Buat embeddings dengan normalisasi
            batch_embeddings = self.model.encode(
                batch_texts,
                normalize_embeddings=True,  # ⭐ PENTING untuk reduce halusinasi
                show_progress_bar=False,
                convert_to_tensor=False
            )
            
            # Tambah metadata
            for j, chunk in enumerate(batch_chunks):
                chunk["embedding"] = batch_embeddings[j].tolist()
                chunk["embedding_model"] = self.model_name
                chunk["chunk_id"] = f"chunk_{i+j}"
                chunk["embedding_version"] = "v2_optimized"
            
            all_embeddings.extend(batch_chunks)
        
        # Simpan hasil
        self._save_embeddings(all_embeddings, output_path)
        return all_embeddings
    
    def _save_embeddings(self, embeddings, output_path):
        """Save embeddings ke file JSONL"""
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        with open(output_path, "w", encoding="utf-8") as f:
            for item in embeddings:
                f.write(json.dumps(item, ensure_ascii=False) + "\n")
        print(f"💾 Embeddings saved to: {output_path}")
        print(f"📊 Total embeddings: {len(embeddings)}")

def main():
    # 🔧 CONFIGURATION - GANTI DI SINI UNTUK COBA MODEL BERBEDA
    MODEL_CONFIGS = {
        "gte-base": "thenlper/gte-base",  # Recommended
        "mpnet-base": "all-mpnet-base-v2",
        "multilingual-e5": "intfloat/multilingual-e5-base", 
        "multilingual-mpnet": "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
        "minilm": "all-MiniLM-L12-v2"  # Lightweight
    }
    
    # Pilih model yang mau dicoba
    SELECTED_MODEL = "gte-base"  # 🔄 GANTI INI UNTUK COBA MODEL LAIN
    
    # Path files
    CHUNKS_PATH = r"D:/chatbot_covid19_intern_procodecg/data/processed/knowladge_covid19_indonesia.jsonl"
    OUTPUT_PATH = r"D:/chatbot_covid19_intern_procodecg/vector_db/embeddings_optimized.jsonl"
    
    print(f"🎯 Using model: {SELECTED_MODEL} -> {MODEL_CONFIGS[SELECTED_MODEL]}")
    
    # Initialize generator
    generator = COVID19EmbeddingGenerator(MODEL_CONFIGS[SELECTED_MODEL])
    
    # Create embeddings
    embeddings = generator.create_embeddings(
        chunks_path=CHUNKS_PATH,
        output_path=OUTPUT_PATH,
        batch_size=8  # Adjust berdasarkan GPU memory
    )
    
    print("✅ Embedding generation completed!")

if __name__ == "__main__":
    main()